In [6]:
import pandas
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from genderize import Genderize

"""
# Not really working
# From 'Song for Bob Dylan', extracted the name 'Song' :(
import spacy
import spacy.cli
spacy.cli.download('en_core_web_lg')
en = spacy.load('en_core_web_lg')

"""


"\n# Not really working\n# From 'Song for Bob Dylan', extracted the name 'Song' :(\nimport spacy\nimport spacy.cli\nspacy.cli.download('en_core_web_lg')\nen = spacy.load('en_core_web_lg')\n\n"

In [19]:
with open('vars.env', 'r') as f:
    my_id = f.readline().rstrip('\n')
    my_secret = f.readline().rstrip('\n')
    playlistURI = f.readline().rstrip('\n')
    owner = f.readline().rstrip('\n')

client_credentials_manager = SpotifyClientCredentials(client_id=my_id, client_secret=my_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlistID = playlistURI[-22:]

In [14]:
results = sp.user_playlist(owner, playlist_id=playlistID, fields='tracks, next')['tracks']

songs =  results['items']
while results['next']:
    results = sp.next(results)
    songs.extend(results['items'])

idSet = {track['added_by']['id'] for track in songs}    
us = {(id, sp.user(id)['display_name']) for id in idSet}
us = dict(us)
us['rmb101'] = 'rmb101'

print(len(songs))


SpotifyOauthError: Bad Request

In [0]:
table = [(track['track']['name'], track['track']['artists'][0]['name'], us[track['added_by']['id']]) 
         for track in songs]
 
df = pandas.DataFrame(table, columns = ['Song', 'Artist', 'Added by'])
df['Song'].replace(' - .*','', regex = True, inplace = True)

df.head()

,Song,Artist,Added by
0,Billie Jean,Michael Jackson,Diana Be
1,Ziggy Stardust,David Bowie,Tom Williams
2,Dirty Diana,Michael Jackson,Diana Be
3,Johnny Too Bad,John Martyn,rjdw500
4,Jolene,Dolly Parton,danielnissenbaum


In [0]:
def gender(song, artist):
    return (Genderize().get(artist.split()))

print(gender('Billie Jean', 'Michael Jackson'))

[{'name': 'Michael', 'gender': 'male', 'probability': 1.0, 'count': 11094}, {'name': 'Jackson', 'gender': 'male', 'probability': 0.98, 'count': 178}]


In [0]:
# Not really working

def entity(text):
    nameList = [name for name in en(text['Song']).ents if name.label_ == 'PERSON']
    if nameList:
        return nameList[0]
    else:
        return None

df['SongNames'] = df.apply(entity, axis = 1)
df.head(50)

,Song,Artist,Added by,SongNames
0,Billie Jean,Michael Jackson,Diana Be,"(Billie, Jean)"
1,Ziggy Stardust,David Bowie,Tom Williams,"(Ziggy, Stardust)"
2,Dirty Diana,Michael Jackson,Diana Be,(Diana)
3,Johnny Too Bad,John Martyn,rjdw500,"(Johnny, Too, Bad)"
4,Jolene,Dolly Parton,danielnissenbaum,(Jolene)
5,Roxanne,The Police,Diana Be,[]
6,Letter To Hermione,David Bowie,Chelle Cole,(Hermione)
7,Rhiannon,Fleetwood Mac,rjdw500,[]
8,Janine,David Bowie,Chelle Cole,(Janine)
9,Maria,Blondie,Greg,[]
